# 智联招聘爬取

## 任务要求

运用所学知识：https://www.zhaopin.com/wuhan/

搜所岗位为爬虫，爬取薪水、任职要求、学历要求，工作年限要求、公司规模，公司名称等数据（“爬取地区为北京的”）。


## 使用 Selenium 库进行爬取

In [1]:
# 导入 selenium 库，以及其他必要的库
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
from tqdm import tqdm

### 1. 初始化Chrome

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.zhaopin.com/wuhan/")
time.sleep(5)

Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)


### 2. 进行登录

In [4]:
# 定位到该按钮
login_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div/div[1]/div/div[2]/div[1]/span[1]/a')
# 进行点击
login_button.click()

### 3. 进行人工扫码登录

### 4. 跳转页面

In [5]:
current_handle = driver.current_window_handle
print(f"当前页面句柄: {current_handle}")
print("Now URL: ", driver.current_url)

当前页面句柄: 4B81C6E1DC03297F4B1619826F2E4F3C
Now URL:  https://www.zhaopin.com/wuhan/


In [6]:
# 由于跳转到了一个新的页面，因此需要进行切换
# 获取当前所有页面的句柄
def switch_new_page():
    all_handles = driver.window_handles
    print(f"所有页面句柄: {all_handles}")

    # 切换到最新的窗口
    driver.switch_to.window(all_handles[-1])
    print("Now URL: ", driver.current_url)
    return all_handles[-1]


In [7]:
switch_new_page()

所有页面句柄: ['4B81C6E1DC03297F4B1619826F2E4F3C', '282A0063F9E6D4ACCD45D4C11CE6C2C5', '6A0172E664A6AABD8D606E239A0FEE8C']
Now URL:  https://i.zhaopin.com/


'6A0172E664A6AABD8D606E239A0FEE8C'

### 5. 搜索 '爬虫'

In [8]:
# 定位到搜索框
search_box = driver.find_element(By.XPATH, '//*[@id="rightNav_top"]/div/div[2]/div/div/div[2]/div/input')
search_box.send_keys("爬虫")

# 定位到搜索按钮
search_button = driver.find_element(By.XPATH, '//*[@id="rightNav_top"]/div/div[2]/div/div/div[2]/button')
search_button.click()

In [9]:
print("Now URL: ", driver.current_url)

Now URL:  https://i.zhaopin.com/


In [10]:
# 切换到最新的页面
switch_new_page()

所有页面句柄: ['4B81C6E1DC03297F4B1619826F2E4F3C', '6A0172E664A6AABD8D606E239A0FEE8C', '9DEBE1847287BE916A3EB6265983E057']
Now URL:  https://www.zhaopin.com/sou/jl736/kwE8M8CQO/p1?srccode=401801


'9DEBE1847287BE916A3EB6265983E057'

### 6. 切换城市为 ‘北京’

In [11]:
flat_button = driver.find_element(By.XPATH, '//*[@id="filter-hook"]/div/div[2]/div/div[1]/a[2]')
flat_button.click()

beijing_label = driver.find_element(By.XPATH, '//*[@id="filter-hook"]/div/div[2]/div[2]/div[1]/ul/li[1]/a')
beijing_label.click()

### 7. 爬取数据

爬取薪水、任职要求、学历要求，工作年限要求、公司规模，公司名称

In [51]:
# 一共有11页
money_ls =  []
require_ls  = []
degree_ls = []
work_year_ls = []
company_scale_ls = []
company_name_ls = []

for i in tqdm(range(1, 12), desc="页面", leave=True):
    # 获取每一个条目
    items = driver.find_elements(By.XPATH, '//*[@id="positionList-hook"]/div/div[1]/div[@class="joblist-box__item clearfix"]')
    print("获取第{}个页面，一共有{}个条目".format(i, len(items)))
    for item in tqdm(items, desc="条目", leave=False):  
        # 前置条件
        pos = 3
        require_exit = item.find_elements(By.CLASS_NAME, 'jobinfo__tag')     # 判断需求标签是否存在
        additional = item.find_elements(By.CLASS_NAME, 'jobinfo__hit-reason')
        if len(require_exit) > 0 and len(additional) > 0:
            pos = 4
        elif len(require_exit) == 0 and len(additional) == 0:
            pos = 2
            
        # 进行休眠
#         time.sleep(random.uniform(2, 3.5))
        # 薪水
        money = item.find_element(By.XPATH, 'div[1]/div[1]/div[1]/p').text
        # 任职要求
        require = ""
        if len(require_exit) > 0:
            requires_temp_ls = item.find_elements(By.XPATH, 'div[1]/div[1]/div[2]/div[@class="joblist-box__item-tag"]')
            for require_temp in requires_temp_ls:
                require += require_temp.text + "|"
        # 学历要求
        degree = item.find_element(By.XPATH, f'div[1]/div[1]/div[{pos}]/div[3]').text
        # 工作年限
        work_year = item.find_element(By.XPATH, f'div[1]/div[1]/div[{pos}]/div[2]').text
        # 公司规模 
        parent_element = item.find_element(By.XPATH, 'div[1]/div[2]/div[2]')
        child_elements = item.find_elements(By.TAG_NAME, 'div')
        if len(child_elements) > 3:
            company_scale = item.find_element(By.XPATH, 'div[1]/div[2]/div[2]/div[2]').text
        else:
            company_scale = item.find_element(By.XPATH, 'div[1]/div[2]/div[2]/div[1]').text
        # 公司名称
        company_name = item.find_element(By.XPATH, 'div[1]/div[2]/div[1]/a').text
        
        # 将一个条目的信息添加到列表中
        money_ls.append(money)
        require_ls.append(require)
        degree_ls.append(degree)
        work_year_ls.append(work_year)
        company_scale_ls.append(company_scale)
        company_name_ls.append(company_name)
    
    # 休眠
    time.sleep(random.uniform(2, 5))
    # 进入下一页 
    parent_next_button = driver.find_element(By.XPATH, '//*[@id="positionList-hook"]/div/div[2]/div[2]/div')
    next_buttons = parent_next_button.find_elements(By.XPATH, './*[last()]')
    if len(next_buttons) > 0:
        driver.execute_script("arguments[0].click();", next_buttons[0])
    time.sleep(5)
print("SUCCESS")

页面:   0%|                                                                                     | 0/11 [00:00<?, ?it/s]

第1个页面，一共有20个条目



页面:   9%|███████                                                                      | 1/11 [00:13<02:15, 13.56s/it]  

第2个页面，一共有20个条目



页面:  18%|██████████████                                                               | 2/11 [00:27<02:03, 13.74s/it]  

...

第10个页面，一共有20个条目



页面:  91%|█████████████████████████████████████████████████████████████████████       | 10/11 [02:08<00:12, 12.54s/it]  

第11个页面，一共有20个条目



页面: 100%|████████████████████████████████████████████████████████████████████████████| 11/11 [02:22<00:00, 12.92s/it]  

SUCCESS


### 8. 查看爬取到的数据

In [52]:
# 输出每一个的最后5条数据
print(money_ls[-5:])
print(require_ls[-5:])
print(degree_ls[-5:])
print(work_year_ls[-5:])
print(company_scale_ls[-5:])
print(company_name_ls[-5:])

['1.5万-1.6万', '2万-4万', '1.5万-2万', '1.9万-2.1万', '1.6万-2万']
['Golang|地图项目|', '航空航天|', 'Python|Django|SQLServer|NoSQL|MySQL|', '', 'Python|WEB|Django|Tornado|Flask|SQLAlchemy|']
['本科', '硕士', '本科', '本科', '大专']
['1-3年', '经验不限', '1-3年', '1-3年', '3-5年']
['1000-9999人', '100-299人', '20-99人', '1000-9999人', '10000人以上']
['首聘(北京)科技有限公司', '北京星际荣耀科技有限责任公司', '北京中科九章软件有限公司', '博彦科技承德有限公司', '软通动力信息技术(集团)股份有限公司']


### 7. 进行持久化存储

保存为 CSV 格式

In [54]:
import pandas as pd

In [55]:
# 构造数据
data = {
    "公司名称": company_name_ls,
    "薪水": money_ls,
    "任职要求": require_ls,
    "学历要求": degree_ls,
    "工作年限要求": work_year_ls,
    "公司规模": company_scale_ls
}
# 转换格式进行存储
df = pd.DataFrame(data)
df.to_csv("智联招聘-爬虫.csv", index=True)

### 8. 退出浏览器

In [12]:
driver.quit()